In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder

In [10]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [11]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)
print(f'X_train shape:{X_train.shape}, X_test shape:{X_test.shape}, y_train shape:{y_train.shape}, y_test shape:{y_test.shape}')

X_train shape:(109820, 16, 3), X_test shape:(27455, 16, 3), y_train shape:(109820,), y_test shape:(27455,)


In [12]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_steps):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=n_input, out_channels=256, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 8, 6)  # Adjusted for the output from the last Conv1d layer
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout(torch.relu(self.conv1(x)))
        x = self.dropout(torch.relu(self.conv2(x)))
        x = self.dropout(torch.relu(self.conv3(x)))
        x = self.dropout(torch.relu(self.conv4(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return self.softmax(x)

# Example instantiation of the model
model = MyModel(n_input=3, n_steps=16)


In [13]:
# Convert arrays to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # Assuming y_train is class labels for classification
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Creating TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Creating DataLoaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [19]:
import torch.optim as optim

# Setting up the device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)  # Reshape input to match model expectations
        #print(f'Input size after reshaping: {inputs.size()}')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.view(-1, 3, 16)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    # Print training/validation statistics
    print(f'Epoch: {epoch+1}/{epochs} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f} \tValidation Accuracy: {val_accuracy:.2f}%')

print('Training complete')


Epoch: 1/20 	Training Loss: 1.2684 	Validation Loss: 1.2809 	Validation Accuracy: 76.27%
Epoch: 2/20 	Training Loss: 1.2580 	Validation Loss: 1.2496 	Validation Accuracy: 79.39%
Epoch: 3/20 	Training Loss: 1.2500 	Validation Loss: 1.2244 	Validation Accuracy: 81.92%
Epoch: 4/20 	Training Loss: 1.2643 	Validation Loss: 1.3078 	Validation Accuracy: 73.58%
Epoch: 5/20 	Training Loss: 1.2414 	Validation Loss: 1.2267 	Validation Accuracy: 81.68%
Epoch: 6/20 	Training Loss: 1.2573 	Validation Loss: 1.2478 	Validation Accuracy: 79.58%
Epoch: 7/20 	Training Loss: 1.2565 	Validation Loss: 1.2556 	Validation Accuracy: 78.80%
Epoch: 8/20 	Training Loss: 1.2516 	Validation Loss: 1.2262 	Validation Accuracy: 81.74%
Epoch: 9/20 	Training Loss: 1.2364 	Validation Loss: 1.2206 	Validation Accuracy: 82.29%
Epoch: 10/20 	Training Loss: 1.2560 	Validation Loss: 1.2815 	Validation Accuracy: 76.21%
Epoch: 11/20 	Training Loss: 1.2893 	Validation Loss: 1.3176 	Validation Accuracy: 72.60%
Epoch: 12/20 	Train

In [26]:
# Test phase - after training is complete
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)
        print(f'Input size after reshaping: {inputs.size()}')
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

# Print test statistics
print(f'Test Loss: {test_loss:.4f} \tTest Accuracy: {test_accuracy:.2f}%')


Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size a

Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size a

Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size a

In [21]:
model_path = "/Users/sandeep/Desktop/BUCourses/Reza_HAR_Project/saved_models/Pytorch/CNN_base.pth"
torch.save(model.state_dict(), model_path)
model.to(device)

MyModel(
  (conv1): Conv1d(3, 256, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(128, 64, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=256, out_features=6, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=1)
)

### Coreml Evaluation

In [34]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for i in range(len(X_test)):
        X_test_sample = X_test[i].view(1, 3, 16)
        #X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_sample.numpy()})
        pred_class = np.argmax(output_dict['var_75'])
        predictions.append(pred_class)
    
    accuracy = np.sum(np.array(predictions) == y_test.numpy()) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

### Convert to coreml model

In [40]:
model_path = "/Users/sandeep/Desktop/BUCourses/Reza_HAR_Project/saved_models/Pytorch/CNN_base.pth"
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [41]:
import coremltools as ct
example_input = torch.rand(1,3,16) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
mlp_coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
mlp_coreml_model.save("cnn.mlpackage")

Running MIL backend_mlprogram pipeline: 100%|███████████████████████████████████████████████████████| 12/12 [00:00<00:00, 13364.75 passes/s]


In [42]:
import coremltools as ct
import coremltools.optimize.coreml as cto

cnn_coreml_model = ct.models.MLModel("cnn.mlpackage")
cnn_coreml_model

input {
  name: "x"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 16
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_75"
  type {
    multiArrayType {
      shape: 1
      shape: 6
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.1.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}

In [43]:
model_name = cnn_coreml_model
model_path = "cnn.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.6763795301402294
Size of the model: 0.12 KB


### Quantization of CNN

In [44]:
import coremltools.optimize.coreml as cto

op_config = cto.OpLinearQuantizerConfig(mode="linear_symmetric", weight_threshold=512)
config = cto.OptimizationConfig(global_config=op_config)

compressed_8_bit_model = cto.linear_quantize_weights(cnn_coreml_model, config=config)


Running compression pass linear_quantize_weights: 100%|███████████████████████████████████████████████████| 9/9 [00:00<00:00, 1122.74 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 4696.43 passes/s]


In [45]:
compressed_8_bit_model.save("gru_8bitQuantized_mlmodel.mlpackage")

In [46]:
model_name = compressed_8_bit_model
model_path = "gru_8bitQuantized_mlmodel.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.6763431069022036
Size of the model: 0.12 KB
